In [1]:
# Sentence Transformer - Based on Colab example
# SiameseBERT SemanticSearch
# 
# https://huggingface.co/sentence-transformers/multi-qa-MiniLM-L6-cos-v1
#
# Nov 10, 2022
# Sila

In [2]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.6 MB/s 
     |████████████████████████████████| 5.5 MB 32.6 MB/s 
     |████████████████████████████████| 1.3 MB 56.5 MB/s 
     |████████████████████████████████| 182 kB 49.7 MB/s 
     |████████████████████████████████| 7.6 MB 46.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=6a43cf0a91e2cf4ff0c5622fdc2e9767108782de2b01df724d749ec7ec7c751a
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [3]:
query = "How many people live in London?"
docs = ["Around 9 Million people live in London", "London is known for its financial district"]

In [5]:
#Load the model
from sentence_transformers import SentenceTransformer
# Load the BERT model
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

# This is a sentence-transformers model: It maps sentences & paragraphs 
# to a 384 dimensional dense vector space and was designed for semantic search. 
# It has been trained on 215M (question, answer) pairs from diverse sources. 

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.5k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/383 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [6]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take average of all tokens
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


#Encode text
def encode(texts):
    # Tokenize sentences
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input, return_dict=True)

    # Perform pooling
    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)
    
    return embeddings


# Sentences we want sentence embeddings for
query = "How many people live in London?"
docs = ["Around 9 Million people live in London", "London is known for its financial district"]

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/multi-qa-MiniLM-L6-cos-v1")
model = AutoModel.from_pretrained("sentence-transformers/multi-qa-MiniLM-L6-cos-v1")

#Encode query and docs
query_emb = encode(query)
doc_emb = encode(docs)

#Compute dot score between query and all document embeddings
scores = torch.mm(query_emb, doc_emb.transpose(0, 1))[0].cpu().tolist()

#Combine docs & scores
doc_score_pairs = list(zip(docs, scores))

#Sort by decreasing score
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

#Output passages & scores
for doc, score in doc_score_pairs:
    print(score, doc)

Downloading:   0%|          | 0.00/383 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

0.9156372547149658 Around 9 Million people live in London
0.49475783109664917 London is known for its financial district


In [33]:
query = "Du er god til at bygge modeller"
docs = ["Du kan lide at konstruere modeller", "Aarhus er en by", "Du er dygtig til at lave modeller", "Du er god til at konstruere modeller", "H.C. Andersen er født i Odense", "Du er dygtig til byggeri og modeller", "Fotomodeller er lige dig", "Du har model konstruktion som dit speciale", "Hun cykler meget", "modeller", "Du er glad for at arbejde med modeller"]

In [34]:
#Encode query and docs
query_emb = encode(query)
doc_emb = encode(docs)

#Compute dot score between query and all document embeddings
scores = torch.mm(query_emb, doc_emb.transpose(0, 1))[0].cpu().tolist()

#Combine docs & scores
doc_score_pairs = list(zip(docs, scores))

#Sort by decreasing score
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

#Output passages & scores
for doc, score in doc_score_pairs:
    print(score, doc)

0.7868664860725403 Du er god til at konstruere modeller
0.7223669290542603 Du er dygtig til byggeri og modeller
0.6555147171020508 Du er dygtig til at lave modeller
0.527798593044281 Du er glad for at arbejde med modeller
0.5096364617347717 modeller
0.49635735154151917 Du kan lide at konstruere modeller
0.4571239650249481 Du har model konstruktion som dit speciale
0.21416670083999634 Hun cykler meget
0.17233005166053772 Aarhus er en by
0.16884134709835052 Fotomodeller er lige dig
0.14215071499347687 H.C. Andersen er født i Odense
